In [1]:
import os
import sys
import time
import glob
import numpy as np
import torch
import utils
from PIL import Image
import logging
import argparse
import torch.utils
import torch.backends.cudnn as cudnn
import torch.nn as nn
from torch.autograd import Variable

from model import *
from multi_read_data import MemoryFriendlyLoader

In [2]:
from torchinfo import summary

In [3]:
in_ = torch.ones(size=(10,3,224,224)) 
sem_ = torch.ones(size=(10,3,224,224)) 
fu_ = torch.ones(size=(10,6,224,224)) 

In [3]:
fm = SemanticFusionUnit(3)

In [6]:
fm(in_,sem_)

tensor([[[[ 7.9064,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [12.3364,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [12.3364,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [12.3364,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [12.3364,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [10.6640,  3.8271,  3.8271,  ...,  3.8271,  3.8271,  5.6366]],

         [[ 4.8029,  9.7964,  9.7964,  ...,  9.7964,  9.7964, 10.7658],
          [ 4.8332,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  6.1045],
          [ 4.8332,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  6.1045],
          ...,
          [ 4.8332,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  6.1045],
          [ 4.8332,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  6.1045],
          [ 5.3385,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 6.1314, 10.2692, 10.2692,  ..., 10.2692, 10.2692, 16.0227],
          [ 0.0000,  0.0000,  

In [4]:
model = Network(stage=3)

In [5]:
model(in_, sem_)

([tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            ...,
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.]],
  
           [[1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            ...,
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.]],
  
           [[1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            ...,
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.]]],
  
  
          [[[1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
         

In [6]:
summary(model, input_size=[(10,3,224,224),(10,3,224,224)], device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
Network                                  [10, 3, 224, 224]         --
├─EnhanceNetwork: 1-1                    [10, 3, 224, 224]         --
│    └─Sequential: 2-1                   [10, 3, 224, 224]         --
│    │    └─Conv2d: 3-1                  [10, 3, 224, 224]         84
│    │    └─ReLU: 3-2                    [10, 3, 224, 224]         --
│    └─SemanticFusionUnit: 2-2           [10, 3, 224, 224]         --
│    │    └─Sequential: 3-3              [10, 3, 224, 224]         171
│    └─ModuleList: 2-17                  --                        (recursive)
│    │    └─Sequential: 3-4              [10, 3, 224, 224]         90
│    └─Sequential: 2-4                   [10, 3, 224, 224]         --
│    │    └─Conv2d: 3-5                  [10, 3, 224, 224]         84
│    │    └─Sigmoid: 3-6                 [10, 3, 224, 224]         --
├─CalibrateNetwork: 1-2                  [10, 3, 224, 224]         --
│    

In [14]:
list(model.named_parameters())

[('enhance.in_conv.0.weight',
  Parameter containing:
  tensor([[[[-0.1275,  0.1306,  0.0903],
            [ 0.1199,  0.0992,  0.1639],
            [-0.0726,  0.1625,  0.0609]],
  
           [[-0.0590,  0.0889, -0.1284],
            [ 0.1278,  0.1884,  0.0352],
            [ 0.0493, -0.1376, -0.1694]],
  
           [[ 0.1659, -0.1453,  0.0175],
            [ 0.1921, -0.1361, -0.0546],
            [ 0.1888, -0.0130,  0.0523]]],
  
  
          [[[ 0.1742, -0.0036,  0.1088],
            [-0.0330, -0.0763,  0.1100],
            [-0.0620, -0.0839, -0.0425]],
  
           [[ 0.1562, -0.0822,  0.0638],
            [-0.0965, -0.0196,  0.0017],
            [ 0.1396,  0.0068, -0.0076]],
  
           [[ 0.1589,  0.1480,  0.1176],
            [ 0.1183,  0.0473, -0.1133],
            [ 0.0633, -0.1543, -0.0777]]],
  
  
          [[[ 0.0651, -0.1766,  0.0139],
            [ 0.0253,  0.1546, -0.1573],
            [ 0.0686,  0.1636,  0.1547]],
  
           [[ 0.0789, -0.1388,  0.1638],
        

In [10]:
name, v = next(model.named_parameters())

In [19]:
np.prod(v.size())

81

In [21]:
torch.rand(1,3).requires_grad

False